# **Implementing YOLO v1 in PyTorch**

<div>
     <center><img src="./img/predictions.jpg" width="750"/> </center>
</div>

## **1. What is YOLO?**

<div>
     <center><img src="./img/yolo.png" width="400"/> </center>
</div>

Developed in 2015 by Redmon et. al., YOLO (You Only Look Once) is a deep learning architecture which greatly improved the efficiency of image detection networks. The goal is simply: to detect objects appearing in an image, with their corresponding bounding boxes (see above image). Previous networks would train separately for category detection (dog, bicyle, laptop, e.g.) and bounding box detection (where an object appears in an image), going back and forth between the two to refine predictions. Redmon et. al. proposed a novel method in which bounding boxes and categories were predicted together in one pass through a single model, hence the model's name *You Only Look Once.*

#### **Why is it an important problem? (SAM)**


#### **Main contributions of YOLO (MATT)**

## **2. YOLO v1 at a High Level**


an intro

#### **Data and Importing (SAM)**

For our training data input, we resize each image to 112px by 112px and apply a greyscale filter. We take the pixel values in a 112x112 matrix and scale the values between -1 and 1. We take all of these scaled image matrices and feed them into our dataloader.
<div>
     <center><img src="./img/imageinput.png" width="750"/> </center>
</div>

#### **Architecture (MATT)**

text

#### **Loss (SAM)**

YOLO v1 implements a custom loss function, described in their paper in the following equation:

<div>
     <center><img src="./img/lossfunction.png" width="750"/> </center>
</div>

This may look overwhelming at first, but it's not overly complicated. The loss function takes the SSE across the predicted $(x,y)$ box midpoint, $(h,w)$ box dimensions, and the set of $c_i$ category probabilities. The first and second line compute SSE loss for the midpoint coordinates and dimensions, the next two lines compute the SSE loss for object confidence (whether an object appears in the box), and the last line computes the SSE for category probability predictions.

**Some important symbols to note:**

$𝟙_{i}^{\textrm{obj}}$ indicates whether an object appears in cell $i$

$𝟙_{ij}^{\textrm{obj}}$ indicates whether the $j^{\textrm{th}}$ bounding box predictor in cell $i$ is "responsible" for that prediction (highest intersection over union, or IOU)

$\lambda_{\textrm{coord}}$ is a learning parameter set to 5

$\lambda_{\textrm{noobj}}$ is a learning parameter set to 0.5

## **3. Our Simplified Implementation**

an intro (MATT)

what we did differently
- from 20 image categories to 5 shape categories
    - changing loss function
- removed bounding_box_2 in loss
- simplified network
    - removed some layers
    - reduced number of kernels per layer (20 vs 200)
- overall compressing output


#### **Data and Importing (MATT)**

text

#### **Architecture (MATT)**

text

#### **Loss (SAM)**

text

#### **Evaluation (MATT)**

text

In [1]:
import torch
import torch.nn

class YOLOLoss(torch.nn.Module):
    """
    Computes loss function according to YOLO v1.

    """
    def __init__(self, S = 7, B = 2, C = 20, l_coord = 5, l_noobj = 0.5):
        super().__init__()
        self.S = S
        self.B = B
        self.C = C
        self.l_coord = l_coord
        self.l_noobj = l_noobj
        self.mse = torch.nn.MSELoss(reduction="sum") # SSE instead of MSE